# Importing Essential Libraries

In [1]:
%matplotlib inline
# from memory_profiler import memory_usage
import IPython.display as ipd
# % pylab inline
import os
import pandas as pd
import numpy as np
import librosa
import glob 
import librosa
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.layers import Convolution2D, MaxPooling2D,  Conv2D, LeakyReLU
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.datasets import make_regression
from sklearn import metrics
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras import regularizers, optimizers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from glob import glob
import librosa
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
import librosa.display
# from path import Path

# Preparing the Data

In [ ]:
filelist = os.listdir("path-to-cat-dataset-train")
cat_files = pd.DataFrame(filelist)
metadata = pd.read_csv("path-to-metadata-file")
filelist = os.listdir("path-to-dog-dataset-train")
dog_files = pd.DataFrame(filelist)

# Merging the Dataset

In [ ]:
cat_files['label']='0'
dog_files['label']='1'
df = pd.concat([dog_files,cat_files],ignore_index=True)
df = df.rename(columns={0:'file'})
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
# Checking Class Balance

df['label'].value_counts()

In [ ]:
# Checking for Erroneous Folders

df[df['file']=='.DS_Store']

# Generating Train and Validation Sets

In [ ]:
train = df[:225]
val = df[225:]

# Generating MelSpectogram for Single File

In [ ]:
filename='drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog/dog_barking_0.wav'
X, sample_rate = librosa.load(filename, sr=None, res_type='kaiser_fast')
S = librosa.feature.melspectrogram(y=X, sr=sample_rate)
librosa.display.specshow(librosa.power_to_db(S, ref=np.max), x_axis='time', y_axis='mel', fmin=50, fmax=280)
plt.show()

# Generating MelSpectogram for Entire Dataset

In [ ]:
# Although this function was modified and many parameteres were explored with, most of it
# came from Source 18 (sources in the READ.ME)

def images(files):
    
    # We define the audiofile from the rows of the dataframe when we iterate through
    # every row of our dataframe for train, val and test
    try:
      audiofile = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat/'+str(files.file)
      X, sample_rate = librosa.load(audiofile, sr=None, res_type='kaiser_fast')
    except FileNotFoundError:
      audiofile = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog/'+str(files.file)
      X, sample_rate = librosa.load(audiofile, sr=None, res_type='kaiser_fast')
    print(audiofile)
   
    # Setting the size of the image
    fig = plt.figure(figsize=[1,1])
    
    # This is to get rid of the axes and only get the picture 
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    
    # This is the melspectrogram from the decibels with a linear relationship
    # Setting min and max frequency to account for human voice frequency
    S = librosa.feature.melspectrogram(y=X, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), x_axis='time', y_axis='mel', fmin=50, fmax=280)
    
    # Here we choose the path and the name to save the file, we will change the path when
    # using the function for train, val and test to make the function easy to use and output
    # the images in different folders to use later with a generator
    name = files.file
    file  = 'drive/MyDrive/ProjectMGN/CatDogTransformed/train/' + str(name) + '.jpg'
    print(file)
    # Here we finally save the image file choosing the resolution 
    plt.savefig(file, dpi=500, bbox_inches='tight',pad_inches=0)
    
    # Here we close the image because otherwise we get a warning saying that the image stays
    # open and consumes memory
    plt.close()

In [ ]:
train.apply(images,axis=1)

In [ ]:
# Although this function was modified and many parameteres were explored with, most of it
# came from Source 18 (sources in the READ.ME)

def images(files):
    
    # We define the audiofile from the rows of the dataframe when we iterate through
    # every row of our dataframe for train, val and test
    try:
      audiofile = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat/'+str(files.file)
      X, sample_rate = librosa.load(audiofile, sr=None, res_type='kaiser_fast')
    except FileNotFoundError:
      audiofile = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog/'+str(files.file)
      X, sample_rate = librosa.load(audiofile, sr=None, res_type='kaiser_fast')
    print(audiofile)
   
    # Setting the size of the image
    fig = plt.figure(figsize=[1,1])
    
    # This is to get rid of the axes and only get the picture 
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    
    # This is the melspectrogram from the decibels with a linear relationship
    # Setting min and max frequency to account for human voice frequency
    S = librosa.feature.melspectrogram(y=X, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), x_axis='time', y_axis='mel', fmin=50, fmax=280)
    
    # Here we choose the path and the name to save the file, we will change the path when
    # using the function for train, val and test to make the function easy to use and output
    # the images in different folders to use later with a generator
    name = files.file
    file  = 'drive/MyDrive/ProjectMGN/CatDogTransformed/test/' + str(name) + '.jpg'
    print(file)
    # Here we finally save the image file choosing the resolution 
    plt.savefig(file, dpi=500, bbox_inches='tight',pad_inches=0)
    
    # Here we close the image because otherwise we get a warning saying that the image stays
    # open and consumes memory
    plt.close()

val.apply(images,axis=1)

In [ ]:
# Function to change the file names to the image names to use them later 
def make_jpg(files):
    return str(files.split('.')[0])+'.wav.jpg'

In [ ]:
train.head()

In [ ]:
train['file']=train['file'].apply(make_jpg)

In [ ]:
train.head()

In [ ]:
val.head()

In [ ]:
val['file']=val['file'].apply(make_jpg)

In [ ]:
val.head()

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
    dataframe=train,
    directory="drive/MyDrive/ProjectMGN/CatDogTransformed/train",
    x_col="file",
    y_col="label",
    batch_size=32,
    shuffle=False,
    class_mode="categorical",
    target_size=(64,64))

In [ ]:
val_generator=datagen.flow_from_dataframe(
    dataframe=val,
    directory="drive/MyDrive/ProjectMGN/CatDogTransformed/test",
    x_col="file",
    y_col="label",
    batch_size=32,
    shuffle=False,
    class_mode="categorical",
    target_size=(64,64))